In [1]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline

def load_p(fname):
    return np.array(pickle.load(open(fname,"rb")))

In [2]:
X_train = load_p("/scratch/pbanerj6/sml-dataset/X_train_cls.p")
X_val = load_p("/scratch/pbanerj6/sml-dataset/X_val_cls.p")
X_test = load_p("/scratch/pbanerj6/sml-dataset/X_test_cls.p")

In [7]:
y_train = load_p("/scratch/pbanerj6/sml-dataset/y_train.p")
y_val = load_p("/scratch/pbanerj6/sml-dataset/y_val.p")
y_test = load_p("/scratch/pbanerj6/sml-dataset/y_test.p")

In [38]:
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import os

In [5]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [8]:
train_dataset = MyDataset(X_train,y_train)
val_dataset = MyDataset(X_val,y_val)
test_dataset = MyDataset(X_test,y_test)

In [36]:
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=20,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=20,
    pin_memory=torch.cuda.is_available()
)

In [28]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 768, 100, 2

In [44]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = nn.CrossEntropyLoss()


In [45]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [46]:
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

patience = 20

max_val_acc = 0.0
for t in tqdm(range(100),desc="Epoch"):
    model.train()
    total_loss = 0.0
    tmp_acc = 0.0
    steps = 0
    nb_eval_examples = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        steps+=1
        logits = model(x)
        loss = loss_fn(logits, y)
        logits = logits.detach().cpu().numpy()
        label_ids = y.to('cpu').numpy()
        tmp_acc += accuracy(logits, label_ids)
        # Compute and print loss.
        
        total_loss += loss.item()
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
        nb_eval_examples += x.size(0)
    
    model.eval()
    val_acc = 0.0
    steps = 0
    nb_val_examples = 0
    for batch_idx, (x, y) in enumerate(val_loader):
        steps+=1
        logits = model(x)
        logits = logits.detach().cpu().numpy()
        label_ids = y.to('cpu').numpy()
        val_acc += accuracy(logits, label_ids)
        nb_val_examples += x.size(0)
    
    if max_val_acc < val_acc/nb_val_examples:
        patience = 20
        print("Saving Best Model!!")
        max_val_acc = val_acc/nb_val_examples
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join("/scratch/pbanerj6/sml-nn-v1", "best_model.bin")
        torch.save(model_to_save.state_dict(), output_model_file)
    else:
        patience-=1
        
    if patience == 0:
        print("Validation Accuracy Did Not Improve After", patience, "Epochs. Stopping.")
        break
    
    
    print("Epoch:",t,"Loss:",total_loss/steps,"Train Accuracy:",(tmp_acc/nb_eval_examples),"Val Accuracy:",(val_acc/nb_val_examples))






Epoch:   0%|          | 0/100 [00:00<?, ?it/s]



Epoch:   1%|          | 1/100 [00:06<10:05,  6.12s/it]

Saving Best Model!!
Epoch: 0 Loss: 3.0374531435205583 Train Accuracy: 0.6661066666666666 Val Accuracy: 0.6308666666666667






Epoch:   2%|▏         | 2/100 [00:12<10:06,  6.19s/it]

Saving Best Model!!
Epoch: 1 Loss: 2.8943424049844135 Train Accuracy: 0.69176 Val Accuracy: 0.6354666666666666






Epoch:   3%|▎         | 3/100 [00:18<10:04,  6.23s/it]

...

Epoch: 28 Loss: 1.9750343447035932 Train Accuracy: 0.8254533333333334 Val Accuracy: 0.6331333333333333






Epoch:  30%|███       | 30/100 [03:14<07:47,  6.68s/it]

Epoch: 29 Loss: 1.9451167130089821 Train Accuracy: 0.83032 Val Accuracy: 0.6379333333333334






Epoch:  31%|███       | 31/100 [03:21<07:40,  6.67s/it]

Epoch: 30 Loss: 1.918594585644438 Train Accuracy: 0.8346533333333334 Val Accuracy: 0.6406
Validation Accuracy Did Not Improve After 0 Epochs. Stopping.


In [47]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
output_model_file = os.path.join("/scratch/pbanerj6/sml-nn-v1", "best_model.bin")
model.load_state_dict(torch.load(output_model_file))
model.eval()

Sequential(
  (0): Linear(in_features=768, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=2, bias=True)
)

In [56]:
from scipy.stats import rankdata
from scipy.special import softmax

def mrrs(out, labels):
    print(out)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
        mrr+=1/rank
    return mrr/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels)

def load_ranking(fname):
    return pickle.load(open("/scratch/pbanerj6/sml-dataset/ranking_"+fname+".p","rb"))

qid2c,qid2indexmap = load_ranking("test")

model.eval()
val_acc = 0.0
steps = 0
nb_val_examples = 0

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=20,
    pin_memory=torch.cuda.is_available()
)

all_preds = []

for batch_idx, (x, y) in enumerate(test_loader):
    logits = model(x)
    logits = logits.detach().cpu().numpy()
    
    for pred in logits:
        all_preds.append(softmax(pred))
    
    label_ids = y.to('cpu').numpy()
    val_acc += accuracy(logits, label_ids)
    nb_val_examples += x.size(0)
    
mrr = mrrwrapper(qid2c,qid2indexmap,all_preds)
print("Test Accuracy:",(val_acc/nb_val_examples),"MRR:",(mrr))

[[0.36931092 0.4086804  0.2969394  0.0818483  0.16670115 0.18603277]
 [0.456922   0.6600688  0.07702929 0.7353013  0.8406557  0.64199984]
 [0.45062605 0.3171125  0.23331673 0.25553235 0.8806134  0.75682116]
 ...
 [0.01513177 0.16570406 0.02787823 0.05568737 0.21851334 0.12771149]
 [0.7124999  0.78986394 0.79848343 0.4941654  0.5839576  0.6593364 ]
 [0.31009448 0.6777135  0.50702024 0.6136262  0.7772534  0.5065988 ]]
Test Accuracy: 0.6520666666666667 MRR: 0.5982222222222253
